In [1]:
'''
The code is used to train BC imitator, or pretrained GAIL imitator
'''

import argparse
import tempfile
import os.path as osp
import gym
import logging
from tqdm import tqdm

import tensorflow as tf

from baselines.gail import mlp_policy
from baselines import bench
from baselines import logger
from baselines.common import set_global_seeds, tf_util as U
from baselines.common.misc_util import boolean_flag

#from baselines.gail.run_mujoco import runner
from baselines.gail.dataset.mujoco_dset import Mujoco_Dset 

##from baselines.common.mpi_adam import MpiAdam

In [7]:
def generate_expert_data_with_rewards(scenario_file, num_interactions=1000, file_name="expert_data", act_ndim=19):
    #from gfrl.base.run_my_ppo2 import create_single_scenic_environment
    import numpy as np
    expert_observations = []
    expert_actions = []
    expert_rewards = []

    gf_env_settings = {
        "stacked": True,
        "rewards": 'scoring',
        "representation": 'extracted',
        "players": [f"agent:left_players=1"],
        "action_set": "default",#"default" "v2"
    }

    from scenic.simulators.gfootball.rl_interface import GFScenicEnv
    from scenic.simulators.gfootball.utilities.scenic_helper import buildScenario
    scenario = buildScenario(scenario_file)
    env = GFScenicEnv(initial_scenario=scenario, gf_env_settings=gf_env_settings, use_scenic_behavior_in_step=True, constraints_checking=True)
    
    obs = env.reset()
    tr = 0
    for i in tqdm(range(num_interactions)):
        expert_observations.append(obs)

        obs, reward, done, info = env.step(env.action_space.sample())
        tr+=reward
        # print(info)
        action = info["action_taken"]
        expert_actions.append(action)

        if done:
            obs = env.reset()
            expert_rewards.append(tr)
            tr = 0

    expert_observations = np.array(expert_observations)
    #expert_observations = np.moveaxis(expert_observations, [3], [1])
    acts = np.array(expert_actions)

    acts_oh = np.zeros((acts.shape[0], act_ndim))
    acts_oh[np.arange(acts.shape[0]), acts] = 1

    expert_rewards = np.array(expert_rewards)
    print("Expert observation shape: ", expert_observations.shape)
    print("Expert actions shape: ", acts_oh.shape)
    print("Num Episode: ", expert_rewards.shape[0])
    print("Mean Reward: ", expert_rewards.mean())

    #np.savez(expert_file_name, obs=mb_obs, acs=mb_act_oh, num_epi = num_episodes, mean_reward = np.sum(mb_rewards)/num_episodes)
    np.savez_compressed(
        file_name,
        acs=acts_oh,
        obs=expert_observations,
        num_epi = expert_rewards.shape[0],
        mean_reward = expert_rewards.mean(),
        rewards = expert_rewards
    )
    return expert_observations, acts_oh, expert_rewards



In [8]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment

scenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"

num_interactions=1000
expert_file_name = f"../_data/pns_{num_interactions}.npz"

obs, act, rew = generate_expert_data_with_rewards(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)

Environment will ignore actions passed to step() and take action provided by Scenic


100%|██████████| 1000/1000 [00:08<00:00, 111.53it/s]


Expert observation shape:  (1000, 72, 96, 16)
Expert actions shape:  (1000, 19)
Num Episode:  27
Mean Reward:  0.6296296296296297


In [9]:
import numpy as np
td = np.load(expert_file_name)

In [10]:
list(td.keys())

['acs', 'obs', 'num_epi', 'mean_reward', 'rewards']

In [11]:
from gfrl.common.mybase.cloning.dataset import GFDset
dataset = GFDset(expert_file_name)
print(dataset.num_epi, dataset.mean_reward, dataset.size, dataset.acts.shape)

27 0.6296296296296297 1000 (1000, 19)


In [15]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment

scenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"

num_interactions=10000
expert_file_name = f"../_data/pns_{num_interactions}.npz"

obs, act, rew = generate_expert_data_with_rewards(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)

  0%|          | 0/10000 [00:00<?, ?it/s]

Environment will ignore actions passed to step() and take action provided by Scenic


100%|██████████| 10000/10000 [02:12<00:00, 75.57it/s]


Expert observation shape:  (10000, 72, 96, 16)
Expert actions shape:  (10000, 19)
Num Episode:  243
Mean Reward:  0.5390946502057613


In [16]:
from gfrl.common.mybase.cloning.dataset import GFDset
dataset = GFDset(expert_file_name)
print(dataset.num_epi, dataset.mean_reward, dataset.size, dataset.acts.shape)

243 0.5390946502057613 10000 (10000, 19)


In [17]:
scenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"

num_interactions=5000
expert_file_name = f"../_data/pns_{num_interactions}.npz"

obs, act, rew = generate_expert_data_with_rewards(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)

Environment will ignore actions passed to step() and take action provided by Scenic


100%|██████████| 5000/5000 [01:35<00:00, 52.43it/s]


Expert observation shape:  (5000, 72, 96, 16)
Expert actions shape:  (5000, 19)
Num Episode:  116
Mean Reward:  0.5517241379310345


In [18]:
from gfrl.common.mybase.cloning.dataset import GFDset
dataset = GFDset(expert_file_name)
print(dataset.num_epi, dataset.mean_reward, dataset.size, dataset.acts.shape)

116 0.5517241379310345 5000 (5000, 19)


In [19]:
scenario = "../_scenarios/academy/wb/pass_n_shoot_wb.scenic"

num_interactions=20000
expert_file_name = f"../_data/pns_{num_interactions}.npz"

obs, act, rew = generate_expert_data_with_rewards(scenario_file=scenario, num_interactions=num_interactions, file_name=expert_file_name)

Environment will ignore actions passed to step() and take action provided by Scenic


100%|██████████| 20000/20000 [14:47<00:00, 22.53it/s]


Expert observation shape:  (20000, 72, 96, 16)
Expert actions shape:  (20000, 19)
Num Episode:  505
Mean Reward:  0.5089108910891089


In [20]:
from gfrl.common.mybase.cloning.dataset import GFDset
dataset = GFDset(expert_file_name)
print(dataset.num_epi, dataset.mean_reward, dataset.size, dataset.acts.shape)

505 0.5089108910891089 20000 (20000, 19)
